In [1]:
import pandas as pd
import numpy as np
import re
import pydit
logger=pydit.setup_logging()


In [2]:
test_list=["Mkey","",1,1,"mkey"]
pydit.deduplicate_list(test_list)

['mkey', 'column_2', '1', '1_2', 'mkey_2']

In [3]:
d1 = {
        1: ["a","b","c","f"],
        "": ["a","b","c","g"],
        "Mkey": [1, 2, 3,4],
        "Mkey": [1, 2, 3,4],
        "Mvalue ": ["a","b","c","d"],
        "Mvalue": ["a","b","c","e"],
        "M value!\nNewLine": ["a","b","c","e"],
        "Mvalue£": ["a","b","c","e"],
        "Mvalue%": ["a","b","c","e"]
        
    }
d2 = {
        "Tkey": [100, 101, 102,103,104,105,106],
        "Mkey": [1,1,1,2,3,3,5],
        "Tvalue": [10.12,20.4,33.3,45,59.99,60,-1]

    }

df1 = pd.DataFrame(data=d1)
df2= pd.DataFrame(data=d2)
df1

,1,,Mkey,Mvalue,Mvalue,M value!\nNewLine,Mvalue£,Mvalue%
0,a,a,1,a,a,a,a,a
1,b,b,2,b,b,b,b,b
2,c,c,3,c,c,c,c,c
3,f,g,4,d,e,e,e,e


In [4]:
pydit.cleanup_column_names(df1)
df1

2022-02-28 20:22 INFO - New columns names:['1', 'column_2', 'mkey', 'mvalue', 'mvalue_2', 'm_value_newline', 'mvalue£', 'mvaluepc']


['1', '', 'mkey', 'mvalue', 'mvalue', 'm_value_newline', 'mvalue£', 'mvaluepc']


,1,column_2,mkey,mvalue,mvalue_2,m_value_newline,mvalue£,mvaluepc
0,a,a,1,a,a,a,a,a
1,b,b,2,b,b,b,b,b
2,c,c,3,c,c,c,c,c
3,f,g,4,d,e,e,e,e
